In [1]:
import os
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs

label_to_name = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
def create_dataset(out_dirpath):
    # 4クラスの点を作成する。
    X, Y = make_blobs(n_samples=1000, n_features=2,
                      centers=[[40, 40], [40, 60], [60, 60], [60, 40]])
    plt.scatter(X[:, 0], X[:, 1], marker='o', c=Y, edgecolor='k')
    plt.show()

    # 点に基づいて、円を描画した画像を生成する。
    for i, (x, y) in enumerate(zip(X, Y)):
        img = np.zeros((100, 100), dtype=np.uint8)
        center = tuple(x.astype(int))
        cv2.circle(img, center, 10, color=255, thickness=-1)

        save_dirpath = os.path.join(out_dirpath, label_to_name[y])
        os.makedirs(save_dirpath, exist_ok=True)

        save_path = os.path.join(save_dirpath, '{}.png'.format(i))
        cv2.imwrite(save_path, img)

# 古いディレクトリ削除
shutil.rmtree('data')
create_dataset('data/train')
create_dataset('data/test')

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [2]:
!tree data -d

data
├── test
│   ├── A
│   ├── B
│   ├── C
│   └── D
└── train
    ├── A
    ├── B
    ├── C
    └── D

10 directories


In [3]:
import glob
import os

import numpy as np
from PIL import Image
from keras.callbacks import CSVLogger
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential, load_model
from keras.utils.np_utils import to_categorical

# ハイパーパラメータ
############################################################
result_dirpath = 'result'  # 結果や重みを出力するディレクトリ
os.makedirs(result_dirpath, exist_ok=True)

batch_size = 32            # バッチサイズ
input_shape = (90, 90, 1)  # 入力の形状 (高さ、幅、チャンネル数)
epochs = 10                # エポック数

Using TensorFlow backend.


In [4]:
# データを作成する。
############################################################

def load_dataset(dirpath):
    '''データセットを読み込む。
    
    以下のディレクトリ構造を想定している。
    data
    |-- test
    |   |-- A
    |   |-- B
    |   |-- C
    |   `-- D
    `-- train
        |-- A
        |-- B
        |-- C
        `-- D
    '''
    data = []           # データ
    labels = []         # ラベル
    label_to_name = {}  # 名前とラベルの対応関係

    for label, label_name in enumerate(sorted(os.listdir(dirpath))):
        label_to_name[label] = label_name

        for img_path in glob.glob(os.path.join(dirpath, label_name, '*.png')):
            # 画像を読み込む。
            img = Image.open(img_path).resize(input_shape[:2])
            x = np.expand_dims(np.array(img), axis=-1)  # (BatchSize, H, W) -> (BatchSize, H, W, 1)
            x = x / 255.  # [0, 255] から [0, 1] に正規化する。

            data.append(x)
            labels.append(label)
    data = np.array(data)
    labels = np.array(labels)

    return data, labels, label_to_name

# データセットを読み込む。
x_train, y_train, label_to_name = load_dataset('data/train')
num_classes = len(label_to_name)  # クラス数

# データセットの情報を表示する。
for label, label_name in label_to_name.items():
    print('class name: {:<5} class id: {} number of images: {}'.format(
        label_name, label, len(x_train[y_train==label])))
# class name: A     class id: 0 number of images: 250
# class name: B     class id: 1 number of images: 250
# class name: C     class id: 2 number of images: 250
# class name: D     class id: 3 number of images: 250

class name: A     class id: 0 number of images: 250
class name: B     class id: 1 number of images: 250
class name: C     class id: 2 number of images: 250
class name: D     class id: 3 number of images: 250


In [5]:
# モデルを作成する。
############################################################
model = Sequential()
model.add(Conv2D(32, 3, activation='relu', input_shape=input_shape))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer='adam',
              metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 88, 88, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 86, 86, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 41, 41, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               12800500  
__________

In [6]:
# ラベルを one-hot 表現に変換する。
onehot_y_train = to_categorical(y_train)

csv_logger = CSVLogger(os.path.join(result_dirpath, 'training.csv'))
history = model.fit(x_train, onehot_y_train, epochs=epochs, callbacks=[csv_logger],
                    batch_size=batch_size, validation_split=0.2)

# モデルを保存する。
model_path = os.path.join(result_dirpath, 'model.h5')
model.save(model_path)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 2s 3ms/step - loss: 0.1208 - acc: 0.9688 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/10
800/800 [==============================] - 1s 795us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/10
800/800 [==============================] - 1s 807us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/10
800/800 [==============================] - 1s 823us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/10
800/800 [==============================] - 1s 806us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/10
800/800 [==============================] - 1s 816us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 7/10
800/800 [==============================] - 1s 807us/step - loss: 1.1921e-07 - acc: 1.0000 - val_l

In [7]:
# モデルを読み込む。
model = load_model(model_path)

# データセットを読み込む。
x_test, y_test, label_to_name = load_dataset('data/test')

# 推論する。
pred = model.predict(x_test)
# 確率値が一番高いラベルを推論結果とする。
y_pred = pred.argmax(axis=1)

# 精度を検証する。
from sklearn.metrics import accuracy_score
accracy = accuracy_score(y_test, y_pred)
print('{:.2%}'.format(accracy))  # 100.00%

100.00%
